In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Tumor detection models : Yolov8

# Table of Contents:
* [Explore and Visulaze Training Data](#chapter1)
* [Tumor Detection With Yolo8](#chapter5)
    * Implement Model
    * Training Model with Tumor Dataset
    * Model evaluation  
* [Conclusion  ](#chapter5)  

In [2]:
from IPython.display import clear_output

In [3]:
#!pip install numpy
#!pip install pandas
#!pip install matplotlib
#!pip install tensorflow
#!pip install seaborn
!pip install -U ultralytics
!pip install pybboxes
!pip install segmentation-models
clear_output()

# Imports  <a class="anchor" id="section1"></a>

In [4]:
#import some necessary librairies
import os
import os.path
from pathlib import Path
import glob
import warnings
import shutil
import itertools
from warnings import filterwarnings
def ignore_warn(*args, **kwargs):
    pass
warnings.warn = ignore_warn #ignore all warning  message from librairies

# linear algebra
import numpy as np
# data processing
import pandas as pd

# plotting
import matplotlib.pyplot as plt
import seaborn as sns

#machine learning
import cv2
from PIL import Image
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report, roc_auc_score, roc_curve
from sklearn.model_selection import train_test_split
import tensorflow as tf
import keras
from keras.models import load_model

In [5]:
from tqdm import tqdm
from google.colab.patches import cv2_imshow

### Configs

In [6]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
#set seed to 42
import tensorflow as tf
tf.random.set_seed(42)
np.random.seed(42)

%matplotlib inline
sns.set_theme()


os.environ['SM_FRAMEWORK'] = 'tf.keras'
tf.config.run_functions_eagerly(True)
import segmentation_models as sm

Segmentation Models: using `tf.keras` framework.


# CONSTANTS

In [7]:
PROJECT_NAME = "Yolov8 Tumor Detection" #just for project title
RANDOM_SEED = 42
IMAGE_SIZE = (640,640)
SAMPLE_NUMBER = 0
# Batch size for training
BATCH_SIZE = 16
# Training Epochs
EPOCHS = 60
CHANNEL= 3
USE_PRETRAINED = False
# Filters for pretrained models
#FILTER_LIST = [ 32,64,128 ]

In [8]:
MAIN_PROJECT_PATH=  "/content/drive/MyDrive/projects/my_final_project/03_yolo_detection_model"
PRETRAINED_MODEL_PATH =    MAIN_PROJECT_PATH+'/runs/detect/yolov8n_fintuend/weights/best.pt'

In [9]:
YOLO_DATASET_PATH = "/content/drive/MyDrive/projects/my_final_project/datasets/yolo_dataset/"
val_images_dir=YOLO_DATASET_PATH+"valid/images"
val_labels_dir=YOLO_DATASET_PATH+"valid/labels"

In [10]:
yaml_file= f"""path: {YOLO_DATASET_PATH}
train: 'train/images'
val:   'valid/images'
test: 'test/images'

nc: 3
names: ['Glioma', 'Meningioma', 'Pituitary']
"""
with open("dataset.yaml","w") as f :
  f.write(yaml_file)

In [11]:
if USE_PRETRAINED:
  %load_ext tensorboard
  %tensorboard --logdir runs/detect/yolov8n_finetuned
else:
  path_to_logs =MAIN_PROJECT_PATH+'/runs/detect/yolov8n_fintuend'
  %load_ext tensorboard
  %tensorboard --logdir path_to_logs

SyntaxError: ignored

In [ ]:
from ultralytics import YOLO

if not USE_PRETRAINED:
    model = YOLO('yolov8n.yaml')
    # Training.
    results = model.train(
      data='dataset.yaml',
      imgsz=640,
      epochs=EPOCHS,
      batch=BATCH_SIZE,
      name='yolov8n_finetuned')
else:
   model=YOLO(PRETRAINED_MODEL_PATH)


                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128

100%|██████████| 755k/755k [00:00<00:00, 38.3MB/s]

Overriding model.yaml nc=80 with nc=3

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

YOLOv8n summary: 225 layers, 3011433 parameters, 3011417 gradients, 8.2 GFLOPs

TensorBoard: Start with 'tensorboard --logdir runs/detect/yolov8n_finetuned', view at http://localhost:6006/
Freezing layer 'model.22.dfl.conv.weight'
AMP: running Automatic Mixed Precision (AMP) checks with YOLOv8n...


100%|██████████| 6.23M/6.23M [00:00<00:00, 193MB/s]


WARNING ⚠️ NMS time limit 0.550s exceeded
AMP: checks passed ✅


# Evaluation model on Test Data

In [ ]:
sample_1=  YOLO_DATASET_PATH+"valid/images/2020_png.rf.61a66efb749331bdac56346e44ca5f50.jpg"
sample_2=  YOLO_DATASET_PATH+"valid/images/2079_png.rf.31eca1b5a5f58ac9b8c9c5095a8b4514.jpg"
sample_3=  YOLO_DATASET_PATH+"valid/images/2095_png.rf.4bc0cde0532e58080addd09486fc4153.jpg"
sample_5=  YOLO_DATASET_PATH+"valid/images/218_png.rf.fb1d030bab9d7fc8aea2d7e19db7cb3c.jpg"
sample_6=  YOLO_DATASET_PATH+"valid/images/855_png.rf.e3ad6e7a65abb542599af8ef0bbb281c.jpg"

### Visulize Trained Models Prediction on Validation images

In [ ]:
def get_image_file_annotation(image_item,dataset_path,file_name):
    import pybboxes as pbx

    points_scaled=None
    full_p = os.path.join(dataset_path.replace("images","labels"),file_name[:-3]+"txt")

    classes = []
    with open(full_p,"r") as f:
      #(x, y, w, h )
      classes =f.read().split("\n")
      for cls in classes:
          yolo_bbox1 =   list(map(float, cls.split(" ")))
          H, W = image_item.shape[:2]
          yolo_bbox1 =yolo_bbox1[1:]
          box_voc=pbx.convert_bbox(yolo_bbox1, from_type="yolo", to_type="voc", image_size=(W, H))
          fname =file_name
          dimensions = image_item.shape
          output_mask=cv2.rectangle(image_item,
                                      (box_voc[0], box_voc[1]),
                                      (box_voc[2], box_voc[3]),
                                      (0, 255, 0), 2)

    return output_mask


### Samples real bbox vs prediction

In [ ]:
img_sample_1 = cv2.imread(sample_1)
img_sample_1=  cv2.resize(img_sample_1,(640,640))
cv2_imshow(get_image_file_annotation(img_sample_1,val_images_dir,sample_1.split("/")[-1]))

In [ ]:
results1 = model(sample_1)  # results list
# Show the results
plt.grid(False)
for r in results1:
    im_array = r.plot()  # plot a BGR numpy array of predictions
    im = Image.fromarray(im_array[..., ::-1])  # RGB PIL image
    plt.imshow(im) # show image


In [ ]:
img_sample_2 = cv2.imread(sample_2)
img_sample_2=  cv2.resize(img_sample_2,(416,416))
cv2_imshow(get_image_file_annotation(img_sample_2,val_images_dir,sample_2.split("/")[-1]))

In [ ]:
results1 = model(sample_2)  # results list
# Show the results
plt.grid(False)
for r in results1:
    im_array = r.plot()  # plot a BGR numpy array of predictions
    im = Image.fromarray(im_array[..., ::-1])  # RGB PIL image
    plt.imshow(im) # show image


### Model Evaluations:

##### Model Confusion Matrix

In [ ]:
PLOTS_PATH = ""
if USE_PRETRAINED:
  PLOTS_PATH=MAIN_PROJECT_PATH+'/runs/detect/yolov8n_finetuned/'
else:
  PLOTS_PATH= '/runs/detect/yolov8n_finetuned/'

In [ ]:
plt.grid(False)
plt.imshow(plt.imread(PLOTS_PATH+"confusion_matrix.png"))

In [ ]:
plt.grid(False)

plt.imshow(plt.imread(PLOTS_PATH+"confusion_matrix_normalized.png"))

##### Model Percesion

In [ ]:
plt.grid(False)
plt.imshow(plt.imread(PLOTS_PATH+"P_curve.png"))

##### Model Recal

In [ ]:
plt.grid(False)
plt.imshow(plt.imread(PLOTS_PATH+"R_curve.png"))

##### Model F1 Score

In [ ]:
plt.grid(False)
plt.imshow(plt.imread(PLOTS_PATH+"F1_curve.png"))

##### Model Results

In [ ]:
plt.figure(figsize=(16,10))
plt.imshow(plt.imread(PLOTS_PATH+"results.png"))

### Evaluation with some Test Data on internet (out of our validation data)

In [ ]:
# Run inference on 'bus.jpg'
results = model("1561fig01.jpeg")  # results list
plt.grid(False)
# Show the results
for r in results:
    im_array = r.plot()  # plot a BGR numpy array of predictions
    im = Image.fromarray(im_array[..., ::-1])  # RGB PIL image
    plt.imshow(im) # show image


In [ ]:
# Run inference on 'bus.jpg'
results = model("big_5b252c1a32b7d.jpg")  # results list
plt.grid(False)
# Show the results
for r in results:
    im_array = r.plot()  # plot a BGR numpy array of predictions
    im = Image.fromarray(im_array[..., ::-1])  # RGB PIL image
    plt.imshow(im) # show image

In [ ]:
import torch
from ultralytics.utils.plot_results import plot_results  # You may need to adjust the import based on YOLOv8's repository structure
from ultralytics.utils.metrics import ap_per_class
from ultralytics.utils.loss import ComputeLoss
from models.experimental import attempt_load

def evaluate_model(weights_path, data_cfg, batch_size, img_size, conf_thres, iou_thres, task='val'):
    # Load model
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    model = attempt_load(weights_path, map_location=device).half()  # Load to FP16 if using GPU
    model.eval()

    # Configure dataset
    data_dict = parse_data_cfg(data_cfg)
    dataset = LoadImagesAndLabels(data_dict[task], batch_size, img_size, augment=False)

    dataloader = torch.utils.data.DataLoader(dataset,
                                             batch_size=batch_size,
                                             num_workers=min([os.cpu_count(), batch_size if batch_size > 1 else 0, 8]),
                                             pin_memory=True,
                                             collate_fn=dataset.collate_fn)

    # Initialize metrics
    seen = 0
    names = model.module.names if hasattr(model, 'module') else model.names
    s = ('%20s' + '%10s' * 7) % ('Class', 'Images', 'Targets', 'P', 'R', 'mAP@.5', 'mAP@.5:.95')
    p, r, f1, mp, mr, map50, map = [0] * 7
    loss = torch.zeros(3)
    jdict, stats, ap, ap_class = [], [], [], []
    compute_loss = ComputeLoss(model)

    # Run through data
    for batch_i, (img, targets, paths, shapes) in enumerate(dataloader):
        img = img.to(device).float() / 255.0
        img = img.to(device).half() if device == 'cuda' else img.to(device)
        targets = targets.to(device)
        nb, _, height, width = img.shape  # Batch size, channels, height, width

        # Forward pass
        with torch.no_grad():
            pred = model(img)[0]

        # Compute loss
        loss += compute_loss(pred, targets, model)[1][:3]  # Box, obj, class losses

        # Compute statistics
        stats += get_batch_statistics(pred, targets, iou_threshold=iou_thres)

        # Log progress
        seen += nb

    # Compute mean metrics
    stats = [np.concatenate(x, 0) for x in list(zip(*stats))]  # to numpy
    if len(stats):
        p, r, ap, f1, ap_class = ap_per_class(*stats)
        mp, mr, map50, map = p.mean(), r.mean(), ap.mean(0), ap[:, 1].mean()

    # Print results
    pf = '%20s' + '%10.3g' * 7  # Print format
    print(pf % ('all', seen, 0, mp, mr, map50, map))

    # Return detailed results
    return (mp, mr, map50, map), model

# Place your paths and configuration here
weights = "/content/drive/MyDrive/projects/my_final_project/03_yolov8_detection_model/runs/detect/yolov8n_finetuned/weights/best.pt"
cfg = 'dataset.yaml'
evaluate_model(weights, cfg, batch_size=16, img_size=640, conf_thres=0.001, iou_thres=0.65)

In [ ]:
results = model.val()

In [ ]:
class_names = list(results.names)
for class_name in class_names:
  class_index = class_names.index(class_name)
  try:
    precision = results.box.class_result(class_index)[0]
    recall = results.box.class_result(class_index)[1]
    f1_score = results.box.class_result(class_index)[2]
    print(f"Class: {class_name}")
    print(f"Precision: {precision}")
    print(f"Recall: {recall}")
    print(f"F1-score: {f1_score}")
  except Exception as e:
    continue

In [ ]:
import pandas as pd

class_names = list(results.names)
metrics_dict = {}
for class_name in class_names:
  try:
    class_index = class_names.index(class_name)
    precision = results.box.class_result(class_index)[0]
    recall = results.box.class_result(class_index)[1]
    f1_score = results.box.class_result(class_index)[2]
    metrics_dict[class_name] = {'Precision': precision, 'Recall': recall, 'F1-score': f1_score}
  except Exception as e:
    continue
df = pd.DataFrame(metrics_dict)
print(df)